In [1]:
!nvidia-smi

Fri Jul 12 02:33:42 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [3]:
!pip install --upgrade accelerate
!pip unistall -y transformers accelerate
!pip install transformers accelerate

ERROR: unknown command "unistall" - maybe you meant "uninstall"


In [1]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset,load_metric
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import torch
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
model_ckpt = "google/pegasus-cnn_dailymail" # model from the hugging face
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
!wget https://github.com/yuehe95/ML-algorithms/raw/main/textsum.zip

--2024-07-12 03:13:09--  https://github.com/yuehe95/ML-algorithms/raw/main/textsum.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/yuehe95/ML-algorithms/main/textsum.zip [following]
--2024-07-12 03:13:09--  https://raw.githubusercontent.com/yuehe95/ML-algorithms/main/textsum.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7903594 (7.5M) [application/zip]
Saving to: ‘textsum.zip’

textsum.zip         100%[===================>]   7.54M  --.-KB/s    in 0.02s   

2024-07-12 03:13:09 (322 MB/s) - ‘textsum.zip’ saved [7903594/7903594]



In [26]:
!unzip textsum.zip

Archive:  textsum.zip
  inflating: samsum-test.csv         
  inflating: samsum-train.csv        
  inflating: samsum-validation.csv   
   creating: samsum_dataset/
 extracting: samsum_dataset/dataset_dict.json  
   creating: samsum_dataset/test/
  inflating: samsum_dataset/test/data-00000-of-00001.arrow  
  inflating: samsum_dataset/test/dataset_info.json  
  inflating: samsum_dataset/test/state.json  
   creating: samsum_dataset/train/
  inflating: samsum_dataset/train/data-00000-of-00001.arrow  
  inflating: samsum_dataset/train/dataset_info.json  
  inflating: samsum_dataset/train/state.json  
   creating: samsum_dataset/validation/
  inflating: samsum_dataset/validation/data-00000-of-00001.arrow  
  inflating: samsum_dataset/validation/dataset_info.json  
  inflating: samsum_dataset/validation/state.json  


In [4]:
data_samsum = load_from_disk('samsum_dataset')
data_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [5]:
def convert_examples_to_features(example_batch):
  input_encoding = tokenizer(example_batch['dialogue'], max_length = 1024,truncation = True)
  with tokenizer.as_target_tokenizer():
    target_encoding = tokenizer(example_batch['summary'], max_length = 128,truncation = True)
  return{
      'input_ids':input_encoding['input_ids'],
      'attention_mask':input_encoding['attention_mask'],
      'labels':target_encoding['input_ids']
  }


In [6]:
dataset_samsum_pt = data_samsum.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [7]:
dataset_samsum_pt['train']

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [8]:
#training
from transformers import DataCollatorForSeq2Seq
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model = model_pegasus)

In [9]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir = 'pegasus-samsum',num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01,logging_steps=10,
    evaluation_strategy="steps", eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
trainer = Trainer(model=model_pegasus, args = trainer_args,
                  tokenizer = tokenizer, data_collator = seq2seq_data_collator,
                  train_dataset = dataset_samsum_pt['test'],
                  eval_dataset = dataset_samsum_pt['validation'])
#should be train but don't have that many memory

In [12]:
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=51, training_loss=3.0043694084765864, metrics={'train_runtime': 174.4685, 'train_samples_per_second': 4.694, 'train_steps_per_second': 0.292, 'total_flos': 313450454089728.0, 'train_loss': 3.0043694084765864, 'epoch': 0.9963369963369964})

In [13]:
#evaluation
def generate_batch_sized_chunks(list_of_elements,batch_size):
  'split dataset into smaller batches that we can process simultaneously'
  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i:i+batch_size]

def calculate_metric_on_test_ds(dataset, metric, model,tokenizer,
                                batch_size = 16,device=device,
                                column_text = 'article',
                                column_summary = 'highlights'):
  article_batches = list(generate_batch_sized_chunks(dataset[column_text],batch_size))
  target_batches = list(generate_batch_sized_chunks(dataset[column_summary],batch_size))

  for article_batch, target_batch in tqdm(
      zip(article_batches,target_batches),total = len(article_batches)):
      inputs = tokenizer(article_batch, max_length=1024, truncation=True,
                         padding="max_length",return_tensors='pt')
      summaries = model.generate(input_ids=inputs['input_ids'].to(device),
                                 attention_mask = inputs['attention_mask'].to(device),
                                 length_penalty=0.8,num_beams=8,max_length=128)
      decoded_summaries = [tokenizer.decode(s,skip_special_tokens=True,
                                            clean_up_tokenization_space=True)
      for s in summaries]

      decoded_summaries = [d.replace(""," ") for d in decoded_summaries]
      metric.add_batch(predictions=decoded_summaries, references=target_batch)

  score = metric.compute()
  return score



In [14]:
rouge_name=['rouge1','rouge2','rougeL','rougeLsum']
rouge_metric = load_metric('rouge')

<ipython-input-14-087d33c709f7>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')


The repository for rouge contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/rouge.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] Y


In [17]:
score = calculate_metric_on_test_ds(
    data_samsum['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size=2,
    column_text = 'dialogue', column_summary = 'summary'
)
rouge_dict = dict((rn,score[rn].mid.fmeasure) for rn in rouge_name)

pd.DataFrame(rouge_dict, index = [f'pegasus'])

100%|██████████| 5/5 [00:22<00:00,  4.47s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.019189,0.0,0.019125,0.019027


In [18]:
#save model
model_pegasus.save_pretrained('pegasus-samsum-model')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


In [19]:
#save tokensizer
tokenizer.save_pretrained('tokenizer')

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [20]:
#load tokenizer
tokenizer = AutoTokenizer.from_pretrained("/content/tokenizer")

In [22]:
#prediction
gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length":128}
sample_text = data_samsum['test'][0]['dialogue']
reference = data_samsum['test'][0]['summary']
pipe = pipeline('summarization', model='pegasus-samsum-model', tokenizer=tokenizer)

print('Dialogue:')
print(sample_text)

print('\nReference Summary:')
print(reference)



Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Reference Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.


In [23]:
print('\nModel Summary:')
print(pipe(sample_text,**gen_kwargs)[0]['summary_text'])

Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)



Model Summary:
Amanda: Ask Larry Amanda: He called her last time we were at the park together .<n>Hannah: I'd rather you texted him .<n>Amanda: Just text him .
